In [2]:
from IPython.display import clear_output
import openmdao.api as om
clear_output()
import importlib
import GUI_notebook
importlib.reload(GUI_notebook)
vb = GUI_notebook.init(In)
vb

Box(children=(Box(children=(Btn(children=['Copy cells'], color='blue lighten-1', height='35px', width='250px')…

```python
import openmdao.api as om
import numpy as np


class Component1(om.ExplicitComponent):
    def setup(self):
        self.add_input("x_name", val=np.nan)
        self.add_output("y_name", units="kg")

    def setup_partials(self):
        self.declare_partials("y_name", "x_name")

    def compute(self, inputs, outputs, discrete_inputs=None, discrete_outputs=None):
        x = inputs["x_name"]

        y = x + 1  # [kg] Mass

        outputs["y_name"] = y

    def compute_partials(self, inputs, J, **kwargs):
        x = inputs["x_name"]

        J["y_name", "x_name"] = 1


```

In [ ]:
"""
---Test Text---

#% Component1

x = 2*y + 3*z #[m]
a = np.pi*b + c*1.2e-3 #[kg]

#% Component2

long_variable = a + b #[long_unit]
function_test = np.log(x + np.sin(y)) #[rad]


"""

In [ ]:
"""
---Test Text---
#%% Group1

#% Component1

x = 2*y + 3*z #[m]
a = b + c*1.2e-3 #[kg]

#% Component2

long_variable = a + b
function_test = np.log(x + np.sin(y)) #[rad]

#%% Group2

#% Component3

x = 2*y + 3*z
a = b + c

#% Component4

long_variable = a + b
function_test = np.log(x + np.sin(y))

"""

In [ ]:
# Import
import numpy as np
import math
from math import sin, cos

In [ ]:
# Init
x = 0
z = 1

In [ ]:
# Exclude
"""
some commentary
"""

In [ ]:
#%%% HG1

#%% Group1

#% Component1
y = x + 1
z = y**2 + x

In [ ]:
# Exclude
y

In [ ]:
# Init
b = 3
c = 4

In [ ]:
#%%% HG2
#%% Group2

#% Component2

x = 2*y + 3*z #[m]
a = b + c*1.2e-3 #[kg]

In [ ]:
# Exclude
import numpy as np
a = 0
b = 1

In [ ]:
#% Component3

long_variable = a + b
function_test = np.log(x + np.sin(y)) #[rad]

In [ ]:
# Import
import numpy as np
from math import sin

In [ ]:
# Init
b = 0
c = 1

In [ ]:
#%% Group
#% Component
a = b**2 + 3*c
# a is an intermediate variable, the user can simply delete it if they do not want it as an imput
d = 3*a + 2 #[m]
e = 2*math.sin(a) - np.cos(d) #[rad]

In [ ]:
# Exclude
a, d ,e